In [1]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
traj = pd.read_csv('./input/traj_100000.csv',usecols=['lon','lat','uid','time'])
traj.sort_values(by=['uid','time'],inplace=True)
traj = traj.dropna()
start_time = pd.to_datetime('2023-07-12')
traj['time'] = pd.to_datetime(traj['time'])
traj = traj[traj['time']>=start_time]
traj['timestamp'] = (traj['time'] - start_time).dt.total_seconds()/60
traj = traj[~((traj['timestamp']%(24*60)==0)&(traj['timestamp']/(24*60)>0))]
traj['elat'] = traj.groupby('uid')['lat'].shift(-1)
traj['elon'] = traj.groupby('uid')['lon'].shift(-1)
traj = traj.dropna()
traj['traj'] = traj.apply(lambda x: str(round(x['lon'],4))+str(round(x['lat'],4))+str(round(x['elon'],4))+str(round(x['elat'],4)),axis=1)
traj.drop_duplicates(subset=['traj','uid'],keep='first',inplace=True)
traj.drop(columns=['traj'],inplace=True)
#traj = traj[traj['uid']<=10]


In [3]:


round_num = 3

traj['md5'] = traj.apply(lambda x: str(round(x['lon'],round_num))+','+str(round(x['lat'],round_num))+','+str(round(x['elon'],round_num))+','+str(round(x['elat'],round_num)),axis=1)
# 对md5去重
traj = traj.drop_duplicates(subset=['md5'],keep='first')



In [4]:
import osmnx as ox
import warnings
warnings.filterwarnings('ignore')
G = ox.load_graphml('./data/shanghai_road2.graphml')
hwy_speeds={"residential": 10, "secondary": 15,
                    'primary': 15, "tertiary": 20}
 # 设置路网
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)  # 将路网转换为GeoDataFrame
# 连接两个节点只保留一条边
gdf_edges = gdf_edges.reset_index().drop_duplicates(
    subset=['u', 'v'], keep='first')
gdf_edges['key'] = 0
gdf_edges = gdf_edges.set_index(['u', 'v', 'key'])
# 重新构建路网
# 图属性,这里使用了之前的路网数据的图属性
G = ox.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=G.graph)
# 路网最大强连通子图
G = ox.utils_graph.get_largest_component(G,
                                            strongly=True  # 是否强连通
                                            )
# 计算每条边出行时长
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

import numpy as np
def find_travel_path(G, slon, slat, elon, elat, md5 , shortest=1, weight='travel_time'):
    # 找到两点间的最短出行路径
    # 由给定的坐标获取最近节点
    orig = ox.distance.nearest_nodes(G, X=slon, Y=slat)
    dest = ox.distance.nearest_nodes(G, X=elon, Y=elat)
    if shortest == 1:
        # 找到最短路径
        travel_route = ox.shortest_path(
            G, orig, dest, weight=weight)
    if shortest > 1:
        # 前k最短路径中选择一个
        routes = ox.k_shortest_paths(
            G, orig, dest, k=shortest, weight=weight)
        routes = list(routes)
        travel_route = routes[np.random.choice(range(len(routes)))]
    # 获取路径上的行驶时间
    travel_time = ox.utils_graph.get_route_edge_attributes(
        G, travel_route, attribute='travel_time')
    length = ox.utils_graph.get_route_edge_attributes(
        G, travel_route, attribute='length')
    # 将路径和行驶时间组合成字典
    route = {
        'direct' : False,
        'travel_route': travel_route,
        'travel_time': travel_time,
        'length': length,
        'has_path': len(travel_route) > 1,
        'slon': slon,
        'slat': slat,
        'elon': elon,
        'elat': elat}
    return route
    

In [5]:
gap = 10000
for i in range(0,int(len(traj)/gap)+1,1):
    print(i*gap,(i+1)*gap)
    trajthis = traj[i*gap:(i+1)*gap]

    trajthis['result'] = trajthis.parallel_apply(lambda row: find_travel_path(G,row['lon'],row['lat'],row['elon'],row['elat'],row['md5']),axis=1)

    route_dict = trajthis.set_index('md5')['result'].to_dict()
    import pickle
    with open(f'./cache/route_dict_{i}.pkl','wb') as file:
        pickle.dump(route_dict,file)

0 10000


10000 20000


20000 30000


30000 40000


40000 50000


...

1120000 1130000


1130000 1140000


1140000 1150000


1150000 1160000


1160000 1170000
